In [1]:
import pandas as pd
from difflib import SequenceMatcher
import random

In [2]:
# read files
df = pd.read_csv('totalset.csv', compression = 'gzip')
pd.set_option('display.max_columns', None)
#df = df.head()

In [3]:
df_1 = df[:34808]
df_2 = df[34808:36375]
df_3 = df[36375:40375]
df_4 = df[40375:40594]
df_5 =  df[40594:]

In [4]:
selections = []
n = 10 #samples per dataset

for dataset in [df_1, df_2, df_3, df_4, df_5]:
    docs = dataset.sample(n)
    docs['dataset'] = f'{dataset}'
    selections.append(docs)

sample = pd.concat(selections)

print(len(sample))
print(sample.head())

50
                       identifier  \
23274  ddd:011225803:mpeg21:a0005   
12943  ddd:010900362:mpeg21:a0001   
22754  ddd:010555169:mpeg21:a0001   
20940  ddd:010972401:mpeg21:a0007   
7996   ddd:010634014:mpeg21:a0001   

                                                 gt text  \
23274  PORTUGAEL. Lissabon den 6 January. Men spreeck...   
12943  Wt Romen, den 18 Februarij 1645. DE Fransche A...   
22754  Wt Londen den 19 October 1640. MEn heeft alhie...   
20940  Wt Brugge den 31 dito. De Troupen van sijn Gen...   
7996   VRANCKRYCK. Parys den eersten Juny. Een Fluyts...   

                                                ocr text    CER     WER  \
23274  PORTUGAEL. Liffabon dt.i 6 Jdnuary. Men fprcec...  22.25   50.00   
12943  Wt Romen,den 18 Februarij 1645. Â«esÃŸa,. c $r...  67.60  100.00   
22754  Wt Londen den 19 October 1640. Â©n Beeft aigie...  49.85  116.85   
20940  Wt Brugge den 31 dito. Â©c (Ctoupcn ban Itjn 4...  65.25  120.25   
7996   VRANCKRYCK. i,«,™-J Een Fluytfd

In [5]:
characters = 1000

for i in range(len(selections)):
    selections[i] = selections[i].reset_index(drop=True)
    for j in range(len(selections[i])):
        if len(selections[i]['gt text'][i]) > characters:
            first_page = selections[i]['gt text'][j][0:characters].split(".")
            first_page = '.'.join(first_page[0:-1])
            selections[i]['gt text'][j] = first_page
        if len(selections[i]['ocr text'][i]) > characters:
            first_page = selections[i]['ocr text'][j][0:characters].split(".")
            first_page = '.'.join(first_page[0:-1])
            selections[i]['ocr text'][j] = first_page

C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [6]:
print(selections[3]['gt text'][0])

Biographisch woordenboek der Nederlanden. H. [G. Marcellus Haack] HAACK (G. Marcellus) was een portretschilder, van wien alleen bekend is, dat hij in 1700 leefde. Zie Kramm, Lev. en Werk. der Kunstsch. [Petrus Haack] HAACK (Petrus) werd den 10den October 1747 te Brielle geboren. Na zijne bevordering tot proponent werd hij, in 1769, predikant te Noord-Gouwe, in 1774 te Zwartewaal, in 1777 te Sommelsdijk, in 1782 te Breda, waar hem in 1784 den titel van hoogleeraar in de godgeleerdheid en gewijde geschiedenissen verleend werd. Van daar vertrok hij in 1789 naar Amsterdam en behoorde tot die predikanten, die in 1796 ontslagen werden, om dat zij weigerden den alstoen gevorderden eed van onderwerping aan den toenmaligen regerings-vorm en de belofte van nimmer op eenige wijze ter herstelling van het stadhouderlijk bestuur te zullen medewerken, af te leggen


In [7]:
print(len(selections[3]['ocr text'][0]))

789


In [8]:
def fuzzy_matching(row): # create files with matched sentences

# split GT and OCR-output of each row into sentences (based on full stops).
    gt_sentences = row['gt text'].split(".")
    ocr_sentences = row['ocr text'].split(".")
    
    match_list = []
    
    for sentence in gt_sentences:
        for match in ocr_sentences:
            if len(sentence) >= 4 and len(match) >= 4: 
                if SequenceMatcher(None, sentence, match).ratio() > 0.75:
                    match_list.extend([[sentence, match]])
    
    print('len match_list:', len(match_list))
    
    matched_gt_sentences = []
    matched_ocr_sentences = []
    
    for sentence in match_list:
        matched_gt_sentences.append(sentence[0])
        matched_ocr_sentences.append(sentence[1])

            
    if len(match_list) > 0: # check if there are enough sentences to select randomly
    #    #print('sentlong:', matched_gt_sentences)
        rannum = random.randint(0,len(matched_gt_sentences)-1)
        matched_gt_sentence = matched_gt_sentences[rannum]
        matched_ocr_sentence = matched_ocr_sentences[rannum]
    else: # otherwise note a sentence is missing
        matched_gt_sentence, matched_ocr_sentence = 'None', 'None'
        #print('none')
        
    return matched_gt_sentence, matched_ocr_sentence
    
        
GT_sentences_datasets = []        
OCR_sentences_datasets = []  
samples_per_dataset = 2
    
for dataset in selections:
    dataset.reset_index(drop = True)
    print('DATASET new')
    GT_matched_sentences_list = []
    OCR_matched_sentences_list = []
    counter = 0
    
    i = 0
    while True:
        #print('len:', len(GT_matched_sentences_list))
        if i == samples_per_dataset:
            print('happened')
            i = 0

        # match sentences GT and OCR and create new documents
        #GT_matched_sentences_list = []
        #OCR_matched_sentences_list = []
        print('i', i)
        #print(dataset.iloc[i])
        
        
        matched_gt_sentence, matched_ocr_sentence = fuzzy_matching(dataset.iloc[i])
        print('found:', matched_gt_sentence, matched_ocr_sentence)
        if matched_gt_sentence != 'None' and (matched_gt_sentence not in GT_matched_sentences_list):
            print('appended')
            counter += 1
            GT_matched_sentences_list.append(matched_gt_sentence)
            OCR_matched_sentences_list.append(matched_ocr_sentence)
        if counter == samples_per_dataset:
            break
        i += 1
        print('i again', i)
        print('PASSED')

    print('sampled sentences:', len(GT_matched_sentences_list))
    GT_sentences_datasets.append(GT_matched_sentences_list)
    OCR_sentences_datasets.append(OCR_matched_sentences_list)
    
print('datasets:', len(GT_sentences_datasets))

DATASET new
i 0
len match_list: 2
found: PORTUGAEL PORTUGAEL
appended
i again 1
PASSED
i 1
len match_list: 1
found: Wt Romen, den 18 Februarij 1645 Wt Romen,den 18 Februarij 1645
appended
sampled sentences: 2
DATASET new
i 0
len match_list: 12
found:  Waarom? A  Waarom ? A
appended
i again 1
PASSED
i 1
len match_list: 3
found:  voorgevallen  voorgevallen
appended
sampled sentences: 2
DATASET new
i 0
len match_list: 13
found:  139, is benoemd tot canton-regter te Gorinchem Mr  139, is benoemd tot canton-regter te Gorinchem Mr
appended
i again 1
PASSED
i 1
len match_list: 14
found:  LEYDEN, 12 Augustus  LEYDEN, 12 Augustus
appended
sampled sentences: 2
DATASET new
i 0
len match_list: 5
found:  en Werk  on Werk
appended
i again 1
PASSED
i 1
len match_list: 13
found:  Derde Druk  Derde Druk
appended
sampled sentences: 2
DATASET new
i 0
len match_list: 5
found:  Vanmiddag is op den onbewaakten overweg te EMPEL in Noord‑Bra‑ bant een 60 jarige� wielrijder door een Dieseltrein gegrepen  y i*-

In [9]:
# word alignment
def word_alignment(GT, OCR):
    shortest_list = min([GT, OCR], key=len)
    longest_list = max([GT, OCR], key=len)
    min_ratio = 0.65

    for i in range(len(longest_list)):
        match = False
        #i = -i-1
        # check if there is a direct match:
        try:
            if OCR[i] == '' and GT[i] == '':
                del OCR[i]
                del GT[i]
            if SequenceMatcher(None, GT[i], OCR[i]).ratio() >= min_ratio:
                match = True
                continue
            elif match == False:
                # check if there is a match with a word before in OCR:
                try:
                    for j in range(1,10):
                        if i-j >= 0:
                            if j >= 5:
                                if (GT[i] >= 5) or (OCR[i-j] >= 5):
                                    pass
                                else:
                                    continue
                            if SequenceMatcher(None, GT[i], OCR[i-j]).ratio() >= min_ratio:
                                if not (GT[i] == '' and OCR[i-j] == ''):
                                    for k in range(0,j):
                                        OCR.insert(i-j, '')
                                    match = True
                                    continue
                        if i+j <= len(OCR):
                            if j >= 5:
                                if (GT[i] >= 5) or (OCR[i+j] >= 5):
                                    pass
                                else:
                                    continue
                            if SequenceMatcher(None, GT[i], OCR[i+j]).ratio() >= min_ratio:
                                if not (GT[i] == '' and OCR[i+j] == ''):
                                    for k in range(0,j):
                                        GT.insert(i, '')
                                    match = True
                                    continue
                except:
                    pass
        except IndexError:
            min([GT, OCR], key=len).append('')

        if match == False:
            try:
                if SequenceMatcher(None, GT[i], OCR[i]).ratio() >= 0.30:
                    pass
            except:
                pass
            else:
                print('bad match', GT[i], OCR[i])
                print('before forwards match GT:', GT)
                print('before forwards bad match OCR:', OCR)
                if i-1 >= 0:
                    GT.insert(i, '')
                else:
                    GT.insert(0, '')
                if i+1 <= len(GT):
                    OCR.insert(i+1, '')
                else:
                    GT.append('')
                print('after forwards match GT:', GT)
                print('after forwards bad match OCR:', OCR)
                
                continue
    
    print('after forwards GT:', GT)
    print('after forwards OCR:', OCR)
    
    while len(GT) != len(OCR):
        min([GT, OCR], key=len).append('')
    
    if True:
        
        shortest_list = min([GT, OCR], key=len)
        longest_list = max([GT, OCR], key=len)

        for i in range(len(longest_list)):
            i = -i-1
            print('i:', i)
            try:

                if (OCR[i] == '') ^ (GT[i] == ''):

                    continue
                if OCR[i] == ' ' and GT[i] == '':
                    del OCR[i]
                    del GT[i]
                if SequenceMatcher(None, GT[i], OCR[i]).ratio() > 0.65:
     
                    continue
                else:
                    # check if there is a match with a word before in OCR:
                    try:
                        for j in range(1,len(GT)):
                            if j >= 10:
                                extra_ratio = 0.1
                            else:
                                extra_ratio = 0
                            if i-j >= -len(OCR):
                                if j >= 5:
                                    if (len(GT[i]) >= 5) or (len(OCR[i-j]) >=5):
                                        pass
                                    else:
                                        continue
                            if GT[i] == '' or OCR[i-j] == '':
                                continue
                            if SequenceMatcher(None, GT[i], OCR[i-j]).ratio() > 0.65 + extra_ratio:
                                for k in range(0,j):
                                    if i+k < 0:
                                        GT.insert(i+1, '')
                                    else: 
                                        GT.append('')
                                break
                            if i+j <= -1:
                                if j >= 5:
                                    if (GT[i] >= 5) or (OCR[i+j] >=5):
                                        pass
                                    else:
                                        continue
                            if GT[i] == '' or OCR[i+j] == '':
                                continue
                            if SequenceMatcher(None, GT[i], OCR[i+j]).ratio() > 0.65 + extra_ratio:
                                for k in range(0,j):
                                    GT.insert(i, '')

                                break
                    except:
                        pass

            except IndexError:
                min([GT, OCR], key=len).append('')

            # if there is no match:
            if SequenceMatcher(None, GT[i], OCR[i]).ratio() > 0.30:
                pass
            else:
                print('bad match', GT[i], OCR[i])
                print('before backwards bad match GT:',  GT)
                print('before backwards bad match OCR:',  OCR)
                if (OCR[i] == '') or (GT[i] == ''):
                    continue
                else:
                    GT.insert(i, '')
                    OCR.insert(i+1, '')
                print('after backwards bad match GT:',  GT)
                print('after backwards bad match OCR:',  OCR)
                
                
    print('after backwards GT:',  GT)
    print('after backwards OCR:',  OCR)
                    
    while len(GT) != len(OCR):
        min([GT, OCR], key=len).append('')


    for i in range(len(GT)-1):
        try:
            print(i)
            print(GT[i], OCR[i])
            if (GT[i] == '') and (OCR[i] == ''):
                del GT[i]
                del OCR[i]
        except:
            break
    
    print('end GT:',  GT)
    print('end OCR:',  OCR)
    return GT, OCR



In [10]:
# word alignment
aligned_datasets_GT = []
aligned_datasets_OCR = []

for i in range(len(GT_sentences_datasets)): # for each dataset
    aligned_dataset_GT = []
    aligned_dataset_OCR = []
    for j in range(len(GT_sentences_datasets[i])): # for each sampled sentences in the dataset
        print('GT_sentences_datasets[i][j]:', GT_sentences_datasets[i][j])
        aligned_GT, aligned_OCR = word_alignment(GT_sentences_datasets[i][j].split(' '), OCR_sentences_datasets[i][j].split(' '))
        aligned_dataset_GT.append(aligned_GT)
        aligned_dataset_OCR.append(aligned_OCR) 
    aligned_datasets_GT.append(aligned_dataset_GT)
    aligned_datasets_OCR.append(aligned_dataset_OCR)

print(len(aligned_datasets_GT))
print(len(aligned_datasets_GT[i]))

GT_sentences_datasets[i][j]: PORTUGAEL
after forwards GT: ['PORTUGAEL']
after forwards OCR: ['PORTUGAEL']
i: -1
after backwards GT: ['PORTUGAEL']
after backwards OCR: ['PORTUGAEL']
end GT: ['PORTUGAEL']
end OCR: ['PORTUGAEL']
GT_sentences_datasets[i][j]: Wt Romen, den 18 Februarij 1645
bad match den 18
before forwards match GT: ['Wt', 'Romen,', 'den', '18', 'Februarij', '1645']
before forwards bad match OCR: ['Wt', 'Romen,den', '18', 'Februarij', '1645']
after forwards match GT: ['Wt', 'Romen,', '', 'den', '18', 'Februarij', '1645']
after forwards bad match OCR: ['Wt', 'Romen,den', '18', '', 'Februarij', '1645']
bad match den 
before forwards match GT: ['Wt', 'Romen,', '', 'den', '18', 'Februarij', '1645']
before forwards bad match OCR: ['Wt', 'Romen,den', '18', '', 'Februarij', '1645']
after forwards match GT: ['Wt', 'Romen,', '', '', 'den', '18', 'Februarij', '1645']
after forwards bad match OCR: ['Wt', 'Romen,den', '18', '', '', 'Februarij', '1645']
bad match den 
before forwards ma

In [11]:
# verify word alignment quality
def verify_alignment(GT, OCR):
    
    if len(GT) != len(OCR):
        alignment_validation = False
        return alignment_validation
    
    min_word_length = 8 # the higher this number, the more likely a alignment will be seen as valid, thus higher chance at false positive, lower chance at false negative. 
    match_ratio = 0.70
    anchors_list = []
    for i in range(len(GT)):
        if len(GT[i]) >= min_word_length:  
            anchors_list.append(GT[i])
        if len(OCR[i]) >= min_word_length:
            anchors_list.append(OCR[i])
    #print(anchors_list)


    alignment_validation = True

    for anchor in anchors_list:
        #print('ANCHOR:', anchor)
        matches_GT = set()
        matches_OCR = set()
        match = False
        for i in range(len(GT)):
            if (SequenceMatcher(None, GT[i], anchor).ratio() >= 0.65) and (len(GT[i]) >= min_word_length or len(GT[i]) == 0):
                matches_GT.add(i)
                #print('added as GT match:', GT[i])
            if SequenceMatcher(None, OCR[i], anchor).ratio() >= 0.65 and (len(OCR[i]) >= min_word_length or len(OCR[i]) == 0):
                matches_OCR.add(i)
                #print('added as OCR match:', OCR[i])
        #print('anchor:', anchor)
        #print(matches_GT)
        #print(matches_OCR)
        if matches_GT == matches_OCR:
            match = True
        if match == False:
            match = True
            points_set1 = set() # counts when the matched opposite word is '' for OCR
            points_set2 = set() # counts when the matched opposite word is '' for GT
            for index in matches_GT.difference(matches_OCR): # elements in matches_GT that are not in matches_OCR
                if OCR[index] == '':
                    points_set1.add(index)
                if OCR[index] != '' and len(matches_OCR.difference(matches_GT)) != 0:
                    match = False
            for index in matches_OCR.difference(matches_GT): # elements in matches_OCR that are not in matches_GT
                if GT[index] == '':
                    points_set2.add(index)
                if GT[index] != '' and len(matches_GT.difference(matches_OCR)) != 0:
                    match = False
            if len(points_set1.intersection(points_set2)) != 0: # if an anchor words was match in both OCR and GT with an '', it's a bad match.
                match = False
        if match == False:
            for index in (matches_GT.difference(matches_OCR)):
                index += 1
                if index in matches_OCR.difference(matches_GT):
                    if (GT[index-1] == '' or OCR[index-1] == '') and (SequenceMatcher(None, GT[index], OCR[index]).ratio() >= match_ratio):
                        match = True
            for index in (matches_OCR.difference(matches_GT)):
                index += 1
                if index in matches_GT.difference(matches_OCR):
                    if (GT[index] == '' or OCR[index] == '') and (SequenceMatcher(None, GT[index-1], OCR[index-1]).ratio() >= match_ratio):
                        match = True

        if match == False:
            alignment_validation = False

    return alignment_validation

In [12]:
# verify word alignment
good_alignments_datasets = []
bad_alignments_datasets = []

for i in range(len(aligned_datasets_GT)):
    good_alignments = []
    bad_alignments = []
    for j in range(len(aligned_datasets_GT[i])):
        alignment_validation = verify_alignment(aligned_datasets_GT[i][j], aligned_datasets_OCR[i][j])
        if alignment_validation == True:
            good_alignments.append([aligned_datasets_GT[i][j], aligned_datasets_OCR[i][j]])
        if alignment_validation == False:
            bad_alignments.append([aligned_datasets_GT[i][j], aligned_datasets_OCR[i][j]])
    good_alignments_datasets.append(good_alignments)
    bad_alignments_datasets.append(bad_alignments)

for i in range(len(good_alignments_datasets)):
    print('Dataset:', i+1)
    print('Good alignments:', len(good_alignments_datasets[i]))
    print('Bad alignments:', len(bad_alignments_datasets[i]))



Dataset: 1
Good alignments: 1
Bad alignments: 1
Dataset: 2
Good alignments: 2
Bad alignments: 0
Dataset: 3
Good alignments: 2
Bad alignments: 0
Dataset: 4
Good alignments: 2
Bad alignments: 0
Dataset: 5
Good alignments: 2
Bad alignments: 0


In [13]:
for i in range(len(good_alignments_datasets)):
    print(f'DATASET {i+1}:')
    print('GOOD ALIGNMENTS:')
    for j in range(len(good_alignments_datasets[i])):
        print('GT:', good_alignments_datasets[i][j][0])
        print('OCR:', good_alignments_datasets[i][j][1])
    print('BAD ALIGNMENTS:')
    for j in range(len(bad_alignments_datasets[i])):
        print('GT:', bad_alignments_datasets[i][j][0])
        print('OCR:', bad_alignments_datasets[i][j][1])

DATASET 1:
GOOD ALIGNMENTS:
GT: ['PORTUGAEL']
OCR: ['PORTUGAEL']
BAD ALIGNMENTS:
GT: ['Wt', 'Romen,', '', '', 'den', '18', 'Februarij', '1645']
OCR: ['Wt', 'Romen,den', '18', '', '', 'Februarij', '1645', '']
DATASET 2:
GOOD ALIGNMENTS:
GT: ['Waarom?', '', 'A', '']
OCR: ['Waarom', '?', 'A', '']
GT: ['voorgevallen', '']
OCR: ['voorgevallen', '']
BAD ALIGNMENTS:
DATASET 3:
GOOD ALIGNMENTS:
GT: ['139,', 'is', 'benoemd', 'tot', 'canton-regter', 'te', 'Gorinchem', 'Mr', '']
OCR: ['139,', 'is', 'benoemd', 'tot', 'canton-regter', 'te', 'Gorinchem', 'Mr', '']
GT: ['LEYDEN,', '12', 'Augustus', '']
OCR: ['LEYDEN,', '12', 'Augustus', '']
BAD ALIGNMENTS:
DATASET 4:
GOOD ALIGNMENTS:
GT: ['', '', 'en', 'Werk']
OCR: ['on', '', '', 'Werk']
GT: ['Derde', 'Druk', '']
OCR: ['Derde', 'Druk', '']
BAD ALIGNMENTS:
DATASET 5:
GOOD ALIGNMENTS:
GT: ['', '', 'Vanmiddag', 'is', 'op', 'den', 'onbewaakten', 'overweg', 'te', 'EMPEL', 'in', '', '', '', '', 'Noord‑Bra‑', 'bant', '', '', 'een', '', '60', '', '', '', '',

In [14]:
#d = {'aligned gt': [1, 2], 'aligned ocr': [3, 4], 'automatic validation': [5,6], 'manual validation': [7,8], 'dataset': [9,10]}
#alignment_df = pd.DataFrame(data=d)
alignment_df = pd.DataFrame(columns=['aligned gt', 'aligned ocr', 'automatic validation', 'manual validation', 'dataset'])

for i in range(len(good_alignments_datasets)): # for every dataset
    print(f'DATASET {i+1}:')
    print('GOOD ALIGNMENTS:')
    for j in range(len(good_alignments_datasets[i])):
        print('GT:', good_alignments_datasets[i][j][0])
        print('OCR:', good_alignments_datasets[i][j][1])
        alignment_df = alignment_df.append({'aligned gt': good_alignments_datasets[i][j][0], 'aligned ocr': good_alignments_datasets[i][j][1], 'automatic validation': True, 'manual validation': '-', 'dataset': i+1}, ignore_index=True)
    print('BAD ALIGNMENTS:')
    for j in range(len(bad_alignments_datasets[i])):
        print('GT:', bad_alignments_datasets[i][j][0])
        print('OCR:', bad_alignments_datasets[i][j][1])
        alignment_df = alignment_df.append({'aligned gt': bad_alignments_datasets[i][j][0], 'aligned ocr': bad_alignments_datasets[i][j][1], 'automatic validation': False, 'manual validation': '-', 'dataset': i+1}, ignore_index=True)

pd.set_option('display.max_columns', None)
alignment_df

DATASET 1:
GOOD ALIGNMENTS:
GT: ['PORTUGAEL']
OCR: ['PORTUGAEL']
BAD ALIGNMENTS:
GT: ['Wt', 'Romen,', '', '', 'den', '18', 'Februarij', '1645']
OCR: ['Wt', 'Romen,den', '18', '', '', 'Februarij', '1645', '']
DATASET 2:
GOOD ALIGNMENTS:
GT: ['Waarom?', '', 'A', '']
OCR: ['Waarom', '?', 'A', '']
GT: ['voorgevallen', '']
OCR: ['voorgevallen', '']
BAD ALIGNMENTS:
DATASET 3:
GOOD ALIGNMENTS:
GT: ['139,', 'is', 'benoemd', 'tot', 'canton-regter', 'te', 'Gorinchem', 'Mr', '']
OCR: ['139,', 'is', 'benoemd', 'tot', 'canton-regter', 'te', 'Gorinchem', 'Mr', '']
GT: ['LEYDEN,', '12', 'Augustus', '']
OCR: ['LEYDEN,', '12', 'Augustus', '']
BAD ALIGNMENTS:
DATASET 4:
GOOD ALIGNMENTS:
GT: ['', '', 'en', 'Werk']
OCR: ['on', '', '', 'Werk']
GT: ['Derde', 'Druk', '']
OCR: ['Derde', 'Druk', '']
BAD ALIGNMENTS:
DATASET 5:
GOOD ALIGNMENTS:
GT: ['', '', 'Vanmiddag', 'is', 'op', 'den', 'onbewaakten', 'overweg', 'te', 'EMPEL', 'in', '', '', '', '', 'Noord‑Bra‑', 'bant', '', '', 'een', '', '60', '', '', '', '',

,aligned gt,aligned ocr,automatic validation,manual validation,dataset
0,[PORTUGAEL],[PORTUGAEL],True,-,1
1,"[Wt, Romen,, , , den, 18, Februarij, 1645]","[Wt, Romen,den, 18, , , Februarij, 1645, ]",False,-,1
2,"[Waarom?, , A, ]","[Waarom, ?, A, ]",True,-,2
3,"[voorgevallen, ]","[voorgevallen, ]",True,-,2
4,"[139,, is, benoemd, tot, canton-regter, te, Go...","[139,, is, benoemd, tot, canton-regter, te, Go...",True,-,3
5,"[LEYDEN,, 12, Augustus, ]","[LEYDEN,, 12, Augustus, ]",True,-,3
6,"[, , en, Werk]","[on, , , Werk]",True,-,4
7,"[Derde, Druk, ]","[Derde, Druk, ]",True,-,4
8,"[, , Vanmiddag, is, op, den, onbewaakten, over...","[y, i*-;, Vanmiddag, is, op, den, onbowaakten,...",True,-,5
9,"[Tweede, uitzending, voor, de, A, ]","[Tweede, uitzending, voor, de, A, ]",True,-,5


In [15]:
for i in range(len(alignment_df)):
    print('GT:', alignment_df['aligned gt'][i])
    print('OCR:', alignment_df['aligned ocr'][i])
    try:
        for j in range(len(max([alignment_df['aligned gt'][i], alignment_df['aligned ocr'][i]], key=len))):
            print('i:', alignment_df['aligned gt'][i][j], alignment_df['aligned ocr'][i][j])
    except:
        pass
    print('Manual validation:', alignment_df['manual validation'][i])
    # fill in manual validation for given sentence
    while True:
        user_input = input("Manual validation: True/False")
        if user_input == 'True' or user_input == 'False':
            alignment_df['manual validation'][i] = user_input
            break
        else:
            pass

alignment_df

GT: ['PORTUGAEL']
OCR: ['PORTUGAEL']
i: PORTUGAEL PORTUGAEL
Manual validation: -
Manual validation: True/FalseTrue
GT: ['Wt', 'Romen,', '', '', 'den', '18', 'Februarij', '1645']
OCR: ['Wt', 'Romen,den', '18', '', '', 'Februarij', '1645', '']
i: Wt Wt
i: Romen, Romen,den
i:  18
i:  
i: den 
i: 18 Februarij
i: Februarij 1645
i: 1645 
Manual validation: -
Manual validation: True/FalseTrue
GT: ['Waarom?', '', 'A', '']
OCR: ['Waarom', '?', 'A', '']
i: Waarom? Waarom
i:  ?
i: A A
i:  
Manual validation: -
Manual validation: True/FalseTrue
GT: ['voorgevallen', '']
OCR: ['voorgevallen', '']
i: voorgevallen voorgevallen
i:  
Manual validation: -
Manual validation: True/FalseTrue
GT: ['139,', 'is', 'benoemd', 'tot', 'canton-regter', 'te', 'Gorinchem', 'Mr', '']
OCR: ['139,', 'is', 'benoemd', 'tot', 'canton-regter', 'te', 'Gorinchem', 'Mr', '']
i: 139, 139,
i: is is
i: benoemd benoemd
i: tot tot
i: canton-regter canton-regter
i: te te
i: Gorinchem Gorinchem
i: Mr Mr
i:  
Manual validation: -
Manu

,aligned gt,aligned ocr,automatic validation,manual validation,dataset
0,[PORTUGAEL],[PORTUGAEL],True,True,1
1,"[Wt, Romen,, , , den, 18, Februarij, 1645]","[Wt, Romen,den, 18, , , Februarij, 1645, ]",False,True,1
2,"[Waarom?, , A, ]","[Waarom, ?, A, ]",True,True,2
3,"[voorgevallen, ]","[voorgevallen, ]",True,True,2
4,"[139,, is, benoemd, tot, canton-regter, te, Go...","[139,, is, benoemd, tot, canton-regter, te, Go...",True,True,3
5,"[LEYDEN,, 12, Augustus, ]","[LEYDEN,, 12, Augustus, ]",True,True,3
6,"[, , en, Werk]","[on, , , Werk]",True,True,4
7,"[Derde, Druk, ]","[Derde, Druk, ]",True,True,4
8,"[, , Vanmiddag, is, op, den, onbewaakten, over...","[y, i*-;, Vanmiddag, is, op, den, onbowaakten,...",True,True,5
9,"[Tweede, uitzending, voor, de, A, ]","[Tweede, uitzending, voor, de, A, ]",True,True,5


In [16]:
alignment_df

,aligned gt,aligned ocr,automatic validation,manual validation,dataset
0,[PORTUGAEL],[PORTUGAEL],True,True,1
1,"[Wt, Romen,, , , den, 18, Februarij, 1645]","[Wt, Romen,den, 18, , , Februarij, 1645, ]",False,True,1
2,"[Waarom?, , A, ]","[Waarom, ?, A, ]",True,True,2
3,"[voorgevallen, ]","[voorgevallen, ]",True,True,2
4,"[139,, is, benoemd, tot, canton-regter, te, Go...","[139,, is, benoemd, tot, canton-regter, te, Go...",True,True,3
5,"[LEYDEN,, 12, Augustus, ]","[LEYDEN,, 12, Augustus, ]",True,True,3
6,"[, , en, Werk]","[on, , , Werk]",True,True,4
7,"[Derde, Druk, ]","[Derde, Druk, ]",True,True,4
8,"[, , Vanmiddag, is, op, den, onbewaakten, over...","[y, i*-;, Vanmiddag, is, op, den, onbowaakten,...",True,True,5
9,"[Tweede, uitzending, voor, de, A, ]","[Tweede, uitzending, voor, de, A, ]",True,True,5


In [17]:
count = 0
while True:
    print(count)
    count += 1
    if count == 10:
        break

0
1
2
3
4
5
6
7
8
9


In [18]:
print(df_4['gt text'][40375][0:2000]) # cut of last sentence = not complete

Boekbeschouwing. De gemoedsgesteldheid van David, onder zijne verschillende lotgevallen; uit zijne Psalmen opgemaakt, en den Christenen tot stichting voorgesteld, in Leerredenen. Door H. van Heyningen, Predikant te Rijswijk. Iste Deel. Te 's Gravenhage, bij S. de Visser. 1824. In gr. 8vo. 395 Bl. f 3-25. De waardige Schrijver is, als populair Schriftuitlegger, door zijne Bijbeloefeningen reeds genoegzaam bekend. Het tegenwoordig werk is van eenen eenigzins anderen aard; want hier ontvangen wij Leerredenen, en hierover verontschuldigt zich de Schrijver genoegzaam. Wij hebben er 14 in dit eerste deel; nog twee deelen van gelijke grootte zullen volgen. Het onderwerp is inderdaad nuttig. Het is de geschiedenis van David's hart, zoo als dat bij zijne velerlei wisselende omstandigheden en lotgevallen in zijn leven gesteld en werkzaam was. De Psalmen zijn de tolken zijner ziel. Het onderwerp is intusschen niet gemakkelijk: er is hier veel gelijksoortigs, waaruit het moeijelijk was eene keuze 

In [19]:
a_string = df_4['gt text'][40375][0:2000]
split_string = a_string.split(".")

substring = split_string[0:-1]
print('.'.join(substring))

Boekbeschouwing. De gemoedsgesteldheid van David, onder zijne verschillende lotgevallen; uit zijne Psalmen opgemaakt, en den Christenen tot stichting voorgesteld, in Leerredenen. Door H. van Heyningen, Predikant te Rijswijk. Iste Deel. Te 's Gravenhage, bij S. de Visser. 1824. In gr. 8vo. 395 Bl. f 3-25. De waardige Schrijver is, als populair Schriftuitlegger, door zijne Bijbeloefeningen reeds genoegzaam bekend. Het tegenwoordig werk is van eenen eenigzins anderen aard; want hier ontvangen wij Leerredenen, en hierover verontschuldigt zich de Schrijver genoegzaam. Wij hebben er 14 in dit eerste deel; nog twee deelen van gelijke grootte zullen volgen. Het onderwerp is inderdaad nuttig. Het is de geschiedenis van David's hart, zoo als dat bij zijne velerlei wisselende omstandigheden en lotgevallen in zijn leven gesteld en werkzaam was. De Psalmen zijn de tolken zijner ziel. Het onderwerp is intusschen niet gemakkelijk: er is hier veel gelijksoortigs, waaruit het moeijelijk was eene keuze 

In [20]:
print('test')

test


In [21]:
print('a')

a


In [ ]:
while True:
        user_input = input("Manual validation: True/False")
        print(type(user_input))
        if user_input == 'True' or user_input == 'False':
            print(user_input)
            break
        else:
            pass